In [1]:
import datetime
import pandas as pd
import utils

log_path = "evaluator_output/evaluator_log.txt"

datetime_str = datetime.datetime.now().isoformat(sep=" ", timespec="seconds")
utils.log("Experiment " + datetime_str, log_path)

#validate_ids = [3, 11, 39, 43]
example_ids=[2, 10, 38, 42]
validate_ids = [x for x in range(48) if x not in example_ids and x not in range(9, 24)]
utils.log("validation data ids:" + str(validate_ids), log_path)

pd.set_option('display.max_colwidth', None)
df = pd.read_csv('data/evaluator_filtered.csv')
display(df.iloc[validate_ids])

Experiment 2023-12-02 17:08:49
validation data ids:[0, 1, 3, 4, 5, 6, 7, 8, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 43, 44, 45, 46, 47]


,Text,Number of Tokens,Rating out of 3,Rating,Gan's Rating,Yimin's Rating
0,"Q: Is it true that there are two kinds of people serving as deputies of the Supreme Soviet of the USSR, as members of the Supreme Court, and as Soviet diplomats? A: Yes, it is true. One kind is those not capable of anything at all, and the other is those capable of anything whatsoever.",64,3,5,4,5
1,"Q: Why do we need two central newspapers, Pravda (Truth) and Izvestiya (News) if both are organs of the same Party? A: Because in Pravda there is no news, and in Izvestiya there is no truth.",54,3,5,5,4
3,"Q: What is the difference between the Constitutions of the USA and USSR? Both guarantee freedom of speech. A: Yes, and the US Constitution also guarantees freedom after speech.",36,3,5,5,4
4,"Q: Can a son of a General become a Marshal? A: No, because every Marshal also has a son.",24,3,5,5,5
5,"Q: Why did you not broadcast for such a long time? A: We had to make some changes to our staff. The previous broadcaster, while reading an article that contained the words, ""Socialism is nothing as compared with communism,"" made a pause too long after the word ""nothing.""",59,2,4,3,4
6,"Q: Does China have rockets powerful enough to reach the moon? A: They don't need rockets. If the Chinese communist Party ordered, their people just would step on each other's shoulders, and this way they could reach even the sun.",49,2,3,3,2
7,"Q: Why do some people say that Hungarians love the Russians and hate the Americans? A: Because Russians helped Hungarians to get rid of one totalitarian rule, but Americans didn't help to get rid of the other.",45,2,4,4,3
8,"Q: Is it true that American skyscrapers are the tallest in the world? A: Yes, it's true, but on the other hand the Soviet-made transistors are the largest in the world.",43,2,4,3,4
24,"Q: Are the bedbugs also builders of socialism? A: Of course, in their veins flows the workers-and-peasants' blood.",29,2,4,5,2
25,"Q: Why Lenin wore regular shoes, but Stalin wore boots? A: At Lenin's time, Russia was still only ankle-high in shit.",29,2,3,4,2


In [2]:
from evaluator import Evaluator
from sklearn.metrics import classification_report, mean_squared_error

evaluator = Evaluator(verbose=True, log_path=log_path)

ratings = []
explanations = []
for i in validate_ids:
    rating, explanation = evaluator.evaluate(df.iloc[i,0], verbose=True)
    ratings.append(rating)
    explanations.append(explanation)

df2 = df.iloc[validate_ids]
df2.insert(3, 'GPT Rating', ratings)
df2.insert(4, 'GPT Explanation', explanations)
df2.to_csv("evaluator_output/" + datetime_str.replace(" ", "_").replace(":", "-") + ".csv", index=False)

utils.log("rmse: " + str(mean_squared_error(df2.iloc[:,2], df2.iloc[:,3], squared=False)), log_path)
utils.log(classification_report(df2.iloc[:,2], df2.iloc[:,3], labels=range(4), zero_division=0.0), log_path)
utils.log("\n\n", log_path)

model: gpt-4-1106-preview, temperature: 1, top_p: 1
example data ids:[2, 10, 38, 42]
system prompt:
### instruction ###
Act as a Russian political joke evaluator. Think why it is funny or not first, then give an integer rating from 0 to 3, in a format of reason -> rating.
### examples ###
Example #1
<user>: '''Q: Is it true that every Soviet soldier dreams of becoming a general? A: No, our soldiers are not that stupid. They know that generals may become MIA even in time of peace.'''
<assistant>: '''This joke is very funny because it is easy to understand and plays on the absurdities and contradictions of the Soviet regime and its leadership -> 3'''
Example #2
<user>: '''Q: Could an atomic bomb destroy our beloved town, Yerevan, with its splendid buildings and beautiful gardens? A: In principle, yes. But Moscow is by far a more beautiful city.'''
<assistant>: '''This joke is moderately funny since it plays with historal facts in Soviet times, but it requires some knowledge to understand